## Usage

To get started, create a virtual environment and install the required packages by following these steps:

```bash
# Create a virtual environment
python3 -m venv .venv

# Activate the virtual environment
source .venv/bin/activate

# Install the required packages
pip install jupedsim plotly
```


In [6]:
import jupedsim as jps
import shapely
import pathlib
import pedpy
from jupedsim.internal.notebook_utils import animate, read_sqlite_file
from shapely import Polygon
from numpy.random import normal  # normal distribution of free movement speed
from typing import List, Tuple
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

In [7]:
def add_agents_to_simulation(
    simulation: jps.Simulation,
    agent_parameters,
    goals: List[Polygon],
    positions: List[Tuple[float, float]],
    speeds: List[float],
):
    exit_ids = [simulation.add_exit_stage(goal) for goal in goals]
    journey = jps.JourneyDescription(exit_ids)
    journey_id = simulation.add_journey(journey)
    for i, (pos, v0) in enumerate(zip(positions, speeds)):
        exit_id = exit_ids[i % len(exit_ids)]
        if agent_parameters == jps.SocialForceModelAgentParameters:
            simulation.add_agent(
                agent_parameters(
                    journey_id=journey_id,
                    stage_id=exit_id,
                    desiredSpeed=v0,
                    position=pos,
                )
            )
        else:
            simulation.add_agent(
                agent_parameters(
                    journey_id=journey_id,
                    stage_id=exit_id,
                    position=pos,
                    v0=v0,
                )
            )

    return simulation


def plot_simulation_configuration(geometry, starting_positions, exit_areas):
    """Plot setup for visual inspection."""
    walkable_area = pedpy.WalkableArea(geometry)
    axes = pedpy.plot_walkable_area(walkable_area=walkable_area)
    for exit_area in exit_areas:
        axes.fill(*exit_area.exterior.xy, color="indianred")

    axes.scatter(*zip(*starting_positions), label="Starting Position", s=1)
    axes.set_xlabel("x/m")
    axes.set_ylabel("y/m")
    axes.set_aspect("equal")
    axes.grid(True, alpha=0.3)


def run_simulation(
    simulation: jps.Simulation,
    goals: List[Polygon],
    positions: List[Tuple[float, float]],
    speeds: List[float],
    max_iterations: int = 4000,
    frequency: int = 1000,
):
    while (
        simulation.agent_count() > 0 and simulation.iteration_count() < max_iterations
    ):
        simulation.iterate()
        if simulation.iteration_count() % frequency == 0:
            logging.info(
                f"Iteration {simulation.iteration_count()}: {simulation.agent_count()} agents remaining"
            )
            add_agents_to_simulation(
                simulation,
                jps.CollisionFreeSpeedModelV2AgentParameters,
                goals,
                positions,
                speeds,
            )
    logging.info(f"Evacuation time: {simulation.elapsed_time():.2f} s")
    return simulation.elapsed_time()


def define_goals():
    """Define goal polygons."""
    goals = [
        Polygon([(44, 26), (45, 26), (45, 25), (44, 25)]),  # Goal 1
    ]
    return goals


def define_positions(num_agents: int, seed: float):
    spawning_area = Polygon([(11, 14), (15, 14), (15, 16), (11, 16)])
    pos_in_spawning_area = jps.distributions.distribute_by_number(
        polygon=spawning_area,
        number_of_agents=num_agents,
        distance_to_agents=0.4,
        distance_to_polygon=0.3,
        seed=seed,
    )
    return pos_in_spawning_area


geometry_file = "Disney.wkt"
with open(geometry_file) as f:
    geometry_str = f.readline()

geometry = shapely.from_wkt(geometry_str)

In [ ]:
# Config parameters
num_agents = 20
trajectory_file = "disney.sqlite"
speeds = normal(1.34, 0.05, num_agents)
max_iterations=1000000
frequency=1500

goals = define_goals()
positions = define_positions(num_agents=num_agents, seed=42)
plot_simulation_configuration(geometry, positions, goals)

In [ ]:
simulation = jps.Simulation(
    model=jps.CollisionFreeSpeedModelV2(),
    geometry=geometry,
    dt=0.01,
    trajectory_writer=jps.SqliteTrajectoryWriter(
        output_file=pathlib.Path(trajectory_file), every_nth_frame=5
    ),
)
add_agents_to_simulation(
    simulation,
    jps.CollisionFreeSpeedModelV2AgentParameters,
    goals,
    positions,
    speeds,
)
evac_time = run_simulation(
    simulation, goals, positions, speeds, max_iterations=max_iterations, frequency=frequency
)
trajectories, walkable_area = read_sqlite_file(trajectory_file)

In [ ]:
animate(
    trajectories,
    walkable_area,    
    every_nth_frame=10,
)